In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd 

def load_data(File):
  return pd.read_csv(File,header=None,sep = ',',index_col=False)

from sklearn.datasets import load_breast_cancer

#iris = load_breast_cancer()
#data, label = iris.data, iris.target


# Import The Data
original_data= load_data("/content/gdrive/My Drive/Research/OBS-Network-DataSet_2_Aug27.csv")
# Take a Quick Look at the Data Structure
original_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,3,0.822038,0.190381,1000,0.004815,19.031487,19.038129,0.809619,822.03750,177.96250,1440,90324,73128,17196,130066560,105304320,0.146594,0.780936,0.001838,0.023455,'NB-No Block'
1,9,0.275513,0.729111,100,0.004815,72.889036,72.911141,0.270889,27.55125,72.44875,1440,9048,2451,6598,13029120,3529440,0.517669,0.242451,0.002236,0.460725,Block
2,3,0.923707,0.090383,900,0.000633,9.035834,9.038339,0.909617,831.33600,68.66400,1440,81276,73930,7346,117037440,106459200,0.058749,0.886758,0.001751,0.000000,'No Block'
3,9,0.368775,0.637710,100,0.000552,63.737843,63.770999,0.362290,36.87750,63.12250,1440,9048,3278,5770,13029120,4720320,0.522922,0.324522,0.001776,0.439255,Block
4,3,0.905217,0.108670,800,0.000497,10.864208,10.866977,0.891330,724.17375,75.82625,1440,72228,64379,7849,104008320,92705760,0.076069,0.869009,0.001767,0.000000,'No Block'


In [ ]:
# To store purely features data from dataset
data= original_data.iloc[:, :4].values

# To store purely label from dataset
label= original_data.iloc[:,-1]

In [ ]:
# Split the data first Without feature scaling
print("Training_size: {}; Test_size: {}" .format(len(data),len(label)))

Training_size: 569; Test_size: 569


# Method 1

In [ ]:
from sklearn import feature_selection
import sys
import numpy as np
#from sklearn.datasets import load_iris

import time
start = time.clock()

y = label
data = data # shape=(150,4)
print(data.shape)

#计算每个特征的方差
vars = [np.var(data[:,index]) for index in range(len(data[0]))]

sele = feature_selection.VarianceThreshold(threshold=0.3)
X_new = sele.fit_transform(data)#两个fit，这个会返回选择后的数据集
sele.fit(X_new)
print(X_new.shape)
#print(X_new)

end = time.clock()
print(end-start)

(569, 30)
(569, 11)
0.00347799999999987


In [ ]:
from sklearn.model_selection import train_test_split
# splitting the dataset - dataset still contains class label
X_train, X_test, Y_train, Y_test = train_test_split(X_new, y, test_size=0.3, random_state=0)

from sklearn.svm import SVC
svc = SVC().fit(X_train, Y_train)
svc_pre = svc.predict(X_test)
acc_svc = sum(svc_pre == Y_test)/len(Y_test)
print(acc_svc)

0.9298245614035088


# Method 2

In [ ]:
from sklearn import datasets
from sklearn.feature_selection import RFE  #RFECV:带交叉验证的递归特征移除
from sklearn.svm import SVC
start = time.clock()
#dataset = original_data # laod iris dataset

model = SVC(kernel="linear") # build SVC model

rfe = RFE(model,step=1) # limit number of variables to three
rfe = rfe.fit(data,label)
print('对应特征的标记（是否被选中）：')
print(rfe.support_)

end = time.clock()
print(end-start)

对应特征的标记（是否被选中）：
[ True False False False  True  True  True  True  True False False  True
  True False False False False False False False  True  True False False
  True  True  True  True  True False]
2.177455


In [ ]:
import pandas as pd
import numpy as np
#x=[0,2,3,6,7,9,11,12,15,18,22,23,24,25]
X = rfe.support_
X[X==True] = 1
X[X==False] = 0
x=[]
for i in range(0, len(X)):
    if(X[i]==0):
        x.append(i)

df = pd.DataFrame(data)
X_new = df.drop(df.columns[x],axis=1)
#X_new.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
# splitting the dataset - dataset still contains class label
X_train, X_test, Y_train, Y_test = train_test_split(X_new, label, test_size=0.3, random_state=0)

svc = SVC().fit(X_train, Y_train)
svc_pre = svc.predict(X_test)
acc_svc = sum(svc_pre==Y_test)/len(Y_test)
print(acc_svc)

0.9473684210526315


# Method 3

In [ ]:
from sklearn.svm import SVC
# 回归估计器有linear_model.Lasso,分类估计器有linear_model.LogisticRegression和svm.LinearSVC
# SVMs和logistic-regression的API里,参数C控制特征的稀疏性,C越小要筛选剩下的特征越少.
# Lasso的API里,参数alpha越大要筛选剩下的特征越少
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
#df = load_iris()
start = time.clock()
X, y = data,label
#X, y = df.data, df.target
print(X.shape)

svc = SVC(kernel="linear",C=0.1).fit(X, y)

model = SelectFromModel(svc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)
#X_new
end = time.clock()
print(end-start)

(569, 30)
(569, 11)
0.32088300000000025


In [ ]:
from sklearn.model_selection import train_test_split
# splitting the dataset - dataset still contains class label
X_train, X_test, Y_train, Y_test = train_test_split(X_new, y, test_size=0.3, random_state=0)

from sklearn.svm import SVC
svc = SVC().fit(X_train, Y_train)
svc_pre = svc.predict(X_test)
acc_svc = sum(svc_pre==Y_test)/len(Y_test)
print(acc_svc)

0.9649122807017544
